In [1]:
### Packages

import pandas as pd

In [2]:
### CSV-data into a dataframe

def read_data():
    base_path = Path("__file__").parent
    full_path = (base_path / "../data/raw/stackoverflow_raw.csv").resolve()
    stackoverflow = pd.read_csv(os.path.join(full_path))
    return stackoverflow

In [3]:
display(stackoverflow)

NameError: name 'stackoverflow' is not defined